# Multiprocessing - 平行處理

## 學習目標

1. 理解什麼時候適合用 multiprocessing
2. 學習 Pool.map / Pool.starmap 的使用
3. 在訓練和推論中應用平行化
4. 了解共享記憶體和資料傳遞的開銷

## 核心原則

> **Multiprocessing 適合「完全獨立」的任務**。
> 如果任務之間有依賴或需要頻繁通訊，開銷可能超過收益。

In [ ]:
import numpy as np
import time
import multiprocessing as mp
from multiprocessing import Pool, shared_memory
import matplotlib.pyplot as plt

print(f"Available CPU cores: {mp.cpu_count()}")
np.random.seed(42)

---

## 第一部分：Multiprocessing 基礎

### 1.1 Process vs Thread

```
┌─────────────────────────────────────────────────────────────────┐
│                      Threading                                  │
├─────────────────────────────────────────────────────────────────┤
│  ┌─────────┐   ┌─────────┐   ┌─────────┐                       │
│  │ Thread 1│   │ Thread 2│   │ Thread 3│    共享記憶體空間     │
│  └────┬────┘   └────┬────┘   └────┬────┘                       │
│       │             │             │                            │
│       └─────────────┼─────────────┘                            │
│                     ▼                                          │
│            ┌───────────────┐                                   │
│            │     GIL      │  <-- 同一時間只有一個執行          │
│            └───────────────┘                                   │
└─────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────┐
│                    Multiprocessing                              │
├─────────────────────────────────────────────────────────────────┤
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐             │
│  │  Process 1  │  │  Process 2  │  │  Process 3  │             │
│  │  (自己的GIL) │  │  (自己的GIL) │  │  (自己的GIL) │             │
│  │  獨立記憶體  │  │  獨立記憶體  │  │  獨立記憶體  │             │
│  └─────────────┘  └─────────────┘  └─────────────┘             │
│         │               │               │                      │
│         └───────────────┼───────────────┘                      │
│                         ▼                                      │
│                真正的並行執行（多核心）                         │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# 基本使用：Pool.map

def square(x):
    """簡單的任務：計算平方"""
    return x * x

def heavy_computation(x):
    """稍微重一點的任務"""
    # 模擬一些計算
    result = 0
    for i in range(100000):
        result += i * x
    return result

# 測試資料
data = list(range(100))

# Sequential
start = time.perf_counter()
results_seq = [heavy_computation(x) for x in data]
time_seq = time.perf_counter() - start
print(f"Sequential: {time_seq:.3f}s")

# Parallel (4 processes)
start = time.perf_counter()
with Pool(processes=4) as pool:
    results_par = pool.map(heavy_computation, data)
time_par = time.perf_counter() - start
print(f"Parallel (4 processes): {time_par:.3f}s")
print(f"Speedup: {time_seq / time_par:.2f}x")

# 驗證結果相同
print(f"Results match: {results_seq == results_par}")

In [ ]:
# 測試不同進程數的效果

def benchmark_pool_size(func, data, max_processes=None):
    """測試不同進程數的效能"""
    if max_processes is None:
        max_processes = mp.cpu_count()
    
    results = []
    
    # Sequential
    start = time.perf_counter()
    _ = [func(x) for x in data]
    time_seq = time.perf_counter() - start
    results.append((1, time_seq))
    print(f"  1 process (seq): {time_seq:.3f}s")
    
    # Parallel with different numbers of processes
    for n_proc in [2, 4, 8, min(16, max_processes)]:
        if n_proc > max_processes:
            continue
        start = time.perf_counter()
        with Pool(processes=n_proc) as pool:
            _ = pool.map(func, data)
        time_par = time.perf_counter() - start
        results.append((n_proc, time_par))
        print(f"  {n_proc} processes: {time_par:.3f}s (speedup: {time_seq/time_par:.2f}x)")
    
    return results

print("Benchmark with different pool sizes:")
print("=" * 50)
results = benchmark_pool_size(heavy_computation, list(range(100)))

### 1.2 Pool.starmap（多參數）

In [ ]:
def process_with_params(x, multiplier, offset):
    """帶多個參數的函數"""
    result = 0
    for i in range(50000):
        result += i * x * multiplier + offset
    return result

# 準備參數
args_list = [(i, 2, 100) for i in range(50)]  # (x, multiplier, offset)

# Sequential
start = time.perf_counter()
results_seq = [process_with_params(*args) for args in args_list]
time_seq = time.perf_counter() - start
print(f"Sequential: {time_seq:.3f}s")

# Parallel with starmap
start = time.perf_counter()
with Pool(processes=4) as pool:
    results_par = pool.starmap(process_with_params, args_list)
time_par = time.perf_counter() - start
print(f"Parallel (starmap): {time_par:.3f}s")
print(f"Speedup: {time_seq / time_par:.2f}x")

---

## 第二部分：資料傳遞的開銷

**關鍵問題**：Multiprocessing 需要把資料**序列化（pickle）**後傳給子進程，
這有額外的開銷！

如果任務本身很輕量，開銷可能超過收益。

In [ ]:
# 實驗：任務太輕時 multiprocessing 反而慢

def very_light_task(x):
    """非常輕量的任務"""
    return x + 1

def light_task(x):
    """輕量任務"""
    return np.sum(x)

def heavy_task(x):
    """重量任務：矩陣運算"""
    return np.sum(np.dot(x, x.T))

# 測試不同權重的任務
print("Task complexity vs Multiprocessing benefit:")
print("=" * 60)

# Very light task
data_light = list(range(10000))

start = time.perf_counter()
_ = [very_light_task(x) for x in data_light]
time_seq = time.perf_counter() - start

start = time.perf_counter()
with Pool(4) as pool:
    _ = pool.map(very_light_task, data_light)
time_par = time.perf_counter() - start

print(f"Very light task (x+1):")
print(f"  Sequential: {time_seq:.4f}s")
print(f"  Parallel:   {time_par:.4f}s")
print(f"  Speedup:    {time_seq/time_par:.2f}x {'(slower!)' if time_par > time_seq else ''}")

# Heavy task with numpy arrays
data_heavy = [np.random.randn(200, 200).astype(np.float32) for _ in range(100)]

start = time.perf_counter()
_ = [heavy_task(x) for x in data_heavy]
time_seq = time.perf_counter() - start

start = time.perf_counter()
with Pool(4) as pool:
    _ = pool.map(heavy_task, data_heavy)
time_par = time.perf_counter() - start

print(f"\nHeavy task (200x200 matrix multiply):")
print(f"  Sequential: {time_seq:.4f}s")
print(f"  Parallel:   {time_par:.4f}s")
print(f"  Speedup:    {time_seq/time_par:.2f}x")

print("\n結論：")
print("- 任務太輕時，pickle/unpickle 的開銷超過收益")
print("- 任務夠重時，multiprocessing 才有優勢")

---

## 第三部分：圖片處理的平行化

這是 multiprocessing 最典型的應用場景：
- 每張圖片的處理完全獨立
- 處理時間夠長（值得平行化）

In [ ]:
# 模擬圖片預處理 pipeline

def preprocess_image(image):
    """
    模擬圖片預處理流程
    
    包含：
    - 正規化
    - 高斯模糊
    - 邊緣檢測
    """
    # 正規化
    image = (image - image.mean()) / (image.std() + 1e-8)
    
    # 高斯模糊（簡化版）
    kernel = np.array([[1, 2, 1],
                       [2, 4, 2],
                       [1, 2, 1]]) / 16.0
    
    # 卷積
    H, W = image.shape
    blurred = np.zeros((H-2, W-2))
    for i in range(H-2):
        for j in range(W-2):
            blurred[i, j] = np.sum(image[i:i+3, j:j+3] * kernel)
    
    # Sobel 邊緣檢測
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1],
                        [0, 0, 0],
                        [1, 2, 1]])
    
    H2, W2 = blurred.shape
    grad_x = np.zeros((H2-2, W2-2))
    grad_y = np.zeros((H2-2, W2-2))
    
    for i in range(H2-2):
        for j in range(W2-2):
            patch = blurred[i:i+3, j:j+3]
            grad_x[i, j] = np.sum(patch * sobel_x)
            grad_y[i, j] = np.sum(patch * sobel_y)
    
    edges = np.sqrt(grad_x**2 + grad_y**2)
    
    return edges


# 生成測試圖片
n_images = 50
images = [np.random.randn(128, 128).astype(np.float32) for _ in range(n_images)]

print(f"Processing {n_images} images of size 128x128:")
print("=" * 50)

# Sequential
start = time.perf_counter()
results_seq = [preprocess_image(img) for img in images]
time_seq = time.perf_counter() - start
print(f"Sequential:             {time_seq:.3f}s")

# Parallel with different pool sizes
for n_proc in [2, 4, 8]:
    start = time.perf_counter()
    with Pool(processes=n_proc) as pool:
        results_par = pool.map(preprocess_image, images)
    time_par = time.perf_counter() - start
    print(f"Parallel ({n_proc} processes): {time_par:.3f}s (speedup: {time_seq/time_par:.2f}x)")

---

## 第四部分：訓練中的平行化

### 4.1 資料載入平行化

這是最常見的訓練優化：在 GPU 計算時，CPU 並行載入下一批資料。

In [ ]:
class ParallelDataLoader:
    """
    簡易的平行資料載入器
    
    使用 multiprocessing 預載入數據
    """
    
    def __init__(self, data_list, preprocess_fn, batch_size=32, n_workers=4):
        self.data_list = data_list
        self.preprocess_fn = preprocess_fn
        self.batch_size = batch_size
        self.n_workers = n_workers
        
        self.n_samples = len(data_list)
        self.n_batches = (self.n_samples + batch_size - 1) // batch_size
    
    def __iter__(self):
        """生成 batches（平行預處理）"""
        indices = np.random.permutation(self.n_samples)
        
        with Pool(processes=self.n_workers) as pool:
            for i in range(0, self.n_samples, self.batch_size):
                batch_indices = indices[i:i + self.batch_size]
                batch_data = [self.data_list[idx] for idx in batch_indices]
                
                # 平行預處理
                processed = pool.map(self.preprocess_fn, batch_data)
                
                yield np.array(processed)
    
    def __len__(self):
        return self.n_batches


class SequentialDataLoader:
    """對照組：順序載入"""
    
    def __init__(self, data_list, preprocess_fn, batch_size=32):
        self.data_list = data_list
        self.preprocess_fn = preprocess_fn
        self.batch_size = batch_size
        
        self.n_samples = len(data_list)
        self.n_batches = (self.n_samples + batch_size - 1) // batch_size
    
    def __iter__(self):
        indices = np.random.permutation(self.n_samples)
        
        for i in range(0, self.n_samples, self.batch_size):
            batch_indices = indices[i:i + self.batch_size]
            batch_data = [self.data_list[idx] for idx in batch_indices]
            
            # 順序預處理
            processed = [self.preprocess_fn(d) for d in batch_data]
            
            yield np.array(processed)
    
    def __len__(self):
        return self.n_batches

In [ ]:
# 簡化的預處理函數

def simple_preprocess(image):
    """簡單的預處理"""
    # 正規化
    image = (image - image.mean()) / (image.std() + 1e-8)
    
    # 調整大小（模擬）
    # 在實際應用中會用 PIL 或其他庫
    
    # 增加一些計算讓預處理有意義
    for _ in range(5):
        image = np.clip(image, -3, 3)
        image = (image - image.mean()) / (image.std() + 1e-8)
    
    return image


# 生成數據
n_samples = 500
raw_data = [np.random.randn(64, 64).astype(np.float32) for _ in range(n_samples)]

print(f"Data loading benchmark ({n_samples} samples, batch_size=32):")
print("=" * 60)

# Sequential
seq_loader = SequentialDataLoader(raw_data, simple_preprocess, batch_size=32)
start = time.perf_counter()
for batch in seq_loader:
    pass  # 模擬「使用」數據
time_seq = time.perf_counter() - start
print(f"Sequential loading:       {time_seq:.3f}s")

# Parallel with different worker counts
for n_workers in [2, 4, 8]:
    par_loader = ParallelDataLoader(raw_data, simple_preprocess, batch_size=32, n_workers=n_workers)
    start = time.perf_counter()
    for batch in par_loader:
        pass
    time_par = time.perf_counter() - start
    print(f"Parallel ({n_workers} workers): {time_par:.3f}s (speedup: {time_seq/time_par:.2f}x)")

### 4.2 推論平行化

In [ ]:
# 模擬一個簡單的 CNN forward pass

class SimpleCNN:
    """簡單的 CNN（用於測試）"""
    
    def __init__(self, in_channels=1, n_classes=10):
        # 簡化的權重
        self.conv1_w = np.random.randn(16, in_channels, 3, 3).astype(np.float32) * 0.1
        self.conv2_w = np.random.randn(32, 16, 3, 3).astype(np.float32) * 0.1
        self.fc_w = np.random.randn(32 * 6 * 6, n_classes).astype(np.float32) * 0.1
    
    def conv2d(self, x, w):
        """簡單的卷積（使用 einsum）"""
        N, C, H, W = x.shape
        C_out, _, kH, kW = w.shape
        out_H = H - kH + 1
        out_W = W - kW + 1
        
        # 使用 stride_tricks
        shape = (N, C, kH, kW, out_H, out_W)
        strides = x.strides[:2] + x.strides[2:4] + x.strides[2:4]
        patches = np.lib.stride_tricks.as_strided(x, shape=shape, strides=strides)
        
        # einsum for convolution
        out = np.einsum('nchwij,ocij->nohw', patches, w)
        return out
    
    def maxpool2d(self, x):
        """2x2 max pooling"""
        N, C, H, W = x.shape
        return x.reshape(N, C, H//2, 2, W//2, 2).max(axis=(3, 5))
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def forward(self, x):
        """Forward pass"""
        # Conv1 + ReLU + Pool
        x = self.conv2d(x, self.conv1_w)
        x = self.relu(x)
        x = self.maxpool2d(x)
        
        # Conv2 + ReLU + Pool
        x = self.conv2d(x, self.conv2_w)
        x = self.relu(x)
        x = self.maxpool2d(x)
        
        # Flatten + FC
        x = x.reshape(x.shape[0], -1)
        x = x @ self.fc_w
        
        return x


# 創建模型
model = SimpleCNN(in_channels=1, n_classes=10)

In [ ]:
# 因為 model 包含 numpy arrays，不能直接 pickle
# 需要把模型權重作為參數傳遞

def inference_single(args):
    """單張圖片推論"""
    image, conv1_w, conv2_w, fc_w = args
    
    # 重建 forward pass
    x = image[np.newaxis]  # (1, C, H, W)
    
    # Conv1
    N, C, H, W = x.shape
    C_out, _, kH, kW = conv1_w.shape
    shape = (N, C, kH, kW, H-kH+1, W-kW+1)
    strides = x.strides[:2] + x.strides[2:4] + x.strides[2:4]
    patches = np.lib.stride_tricks.as_strided(x, shape=shape, strides=strides)
    x = np.einsum('nchwij,ocij->nohw', patches, conv1_w)
    x = np.maximum(0, x)
    x = x.reshape(N, x.shape[1], x.shape[2]//2, 2, x.shape[3]//2, 2).max(axis=(3, 5))
    
    # Conv2
    N, C, H, W = x.shape
    C_out, _, kH, kW = conv2_w.shape
    shape = (N, C, kH, kW, H-kH+1, W-kW+1)
    strides = x.strides[:2] + x.strides[2:4] + x.strides[2:4]
    patches = np.lib.stride_tricks.as_strided(x, shape=shape, strides=strides)
    x = np.einsum('nchwij,ocij->nohw', patches, conv2_w)
    x = np.maximum(0, x)
    x = x.reshape(N, x.shape[1], x.shape[2]//2, 2, x.shape[3]//2, 2).max(axis=(3, 5))
    
    # FC
    x = x.reshape(N, -1)
    x = x @ fc_w
    
    return x[0]  # 返回單張圖片的結果


# 測試推論平行化
n_images = 100
test_images = [np.random.randn(1, 28, 28).astype(np.float32) for _ in range(n_images)]

print(f"Inference benchmark ({n_images} images):")
print("=" * 50)

# Sequential
start = time.perf_counter()
results_seq = []
for img in test_images:
    result = model.forward(img[np.newaxis])
    results_seq.append(result[0])
time_seq = time.perf_counter() - start
print(f"Sequential: {time_seq:.3f}s")

# Parallel
# 準備參數（包含模型權重）
args_list = [(img, model.conv1_w, model.conv2_w, model.fc_w) for img in test_images]

start = time.perf_counter()
with Pool(4) as pool:
    results_par = pool.map(inference_single, args_list)
time_par = time.perf_counter() - start
print(f"Parallel (4 processes): {time_par:.3f}s")
print(f"Speedup: {time_seq / time_par:.2f}x")

# Batched inference (通常更快)
start = time.perf_counter()
batch = np.array(test_images)  # (N, 1, 28, 28)
results_batch = model.forward(batch)
time_batch = time.perf_counter() - start
print(f"Batched (single process): {time_batch:.3f}s")
print(f"\n注意：對於 numpy 運算，batched 通常比 multiprocessing 更快！")
print("因為 numpy 本身就用了 BLAS 多執行緒。")

---

## 第五部分：共享記憶體

為了減少資料傳遞的開銷，可以使用共享記憶體。

In [ ]:
# 使用 shared_memory 避免資料複製

def create_shared_array(shape, dtype=np.float32):
    """創建共享記憶體的 numpy array"""
    size = int(np.prod(shape)) * np.dtype(dtype).itemsize
    shm = shared_memory.SharedMemory(create=True, size=size)
    arr = np.ndarray(shape, dtype=dtype, buffer=shm.buf)
    return arr, shm

def worker_with_shared_mem(args):
    """使用共享記憶體的 worker"""
    shm_name, shape, dtype, start_idx, end_idx = args
    
    # 連接到共享記憶體
    existing_shm = shared_memory.SharedMemory(name=shm_name)
    arr = np.ndarray(shape, dtype=dtype, buffer=existing_shm.buf)
    
    # 處理自己負責的部分
    result = 0
    for i in range(start_idx, end_idx):
        result += np.sum(arr[i] ** 2)
    
    existing_shm.close()
    return result

def demonstrate_shared_memory():
    """展示共享記憶體的使用"""
    # 創建大型數據
    data_shape = (1000, 100, 100)
    
    print(f"Shared memory demonstration:")
    print(f"Data shape: {data_shape}")
    print(f"Data size: {np.prod(data_shape) * 4 / 1024**2:.1f} MB")
    print("=" * 50)
    
    # 創建共享記憶體 array
    shared_arr, shm = create_shared_array(data_shape)
    shared_arr[:] = np.random.randn(*data_shape).astype(np.float32)
    
    try:
        n_processes = 4
        chunk_size = data_shape[0] // n_processes
        
        # 準備參數
        args_list = [
            (shm.name, data_shape, np.float32, i * chunk_size, (i + 1) * chunk_size)
            for i in range(n_processes)
        ]
        
        # 使用共享記憶體
        start = time.perf_counter()
        with Pool(n_processes) as pool:
            results = pool.map(worker_with_shared_mem, args_list)
        time_shared = time.perf_counter() - start
        print(f"With shared memory: {time_shared:.3f}s")
        
        # 對比：不用共享記憶體（每次複製整個 array）
        def worker_copy(args):
            arr, start_idx, end_idx = args
            result = 0
            for i in range(start_idx, end_idx):
                result += np.sum(arr[i] ** 2)
            return result
        
        # 複製整個 array
        regular_arr = np.array(shared_arr)
        args_copy = [
            (regular_arr, i * chunk_size, (i + 1) * chunk_size)
            for i in range(n_processes)
        ]
        
        start = time.perf_counter()
        with Pool(n_processes) as pool:
            results_copy = pool.starmap(worker_copy, args_copy)
        time_copy = time.perf_counter() - start
        print(f"Without shared memory: {time_copy:.3f}s")
        
        print(f"Speedup: {time_copy / time_shared:.2f}x")
        
    finally:
        # 清理共享記憶體
        shm.close()
        shm.unlink()

demonstrate_shared_memory()

---

## 第六部分：最佳實踐總結

In [ ]:
# 決策流程

decision_tree = """
┌─────────────────────────────────────────────────────────────────┐
│               何時使用 Multiprocessing？                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Q1: 任務是否「完全獨立」？                                      │
│      │                                                          │
│      ├── No → 考慮其他方案（threading for I/O, GPU, etc.）      │
│      │                                                          │
│      └── Yes → Q2: 單個任務是否「夠重」？                        │
│                  │                                              │
│                  ├── No → 可能不值得（開銷超過收益）             │
│                  │         試試 batch 處理或向量化               │
│                  │                                              │
│                  └── Yes → Q3: 資料量大嗎？                      │
│                              │                                  │
│                              ├── Yes → 考慮共享記憶體            │
│                              │                                  │
│                              └── No → 直接用 Pool.map           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘

適合 Multiprocessing 的場景：
✓ 圖片預處理（每張獨立，計算量適中）
✓ 資料增強（隨機變換是獨立的）
✓ 批量檔案處理
✓ 超參數搜索（每組參數獨立訓練）

不太適合的場景：
✗ numpy 矩陣運算（已經用 BLAS 多執行緒了）
✗ 很輕量的任務（開銷超過收益）
✗ 需要頻繁通訊的任務
✗ 訓練中的梯度計算（有依賴關係）
"""

print(decision_tree)

In [ ]:
# 最終效能比較

def final_benchmark():
    """綜合效能比較"""
    
    print("Final Performance Comparison")
    print("=" * 70)
    
    # 場景 1：圖片預處理
    print("\n1. Image Preprocessing (50 images, 128x128):")
    images = [np.random.randn(128, 128).astype(np.float32) for _ in range(50)]
    
    start = time.perf_counter()
    _ = [preprocess_image(img) for img in images]
    t_seq = time.perf_counter() - start
    
    start = time.perf_counter()
    with Pool(4) as pool:
        _ = pool.map(preprocess_image, images)
    t_par = time.perf_counter() - start
    
    print(f"   Sequential: {t_seq:.3f}s")
    print(f"   Parallel:   {t_par:.3f}s (speedup: {t_seq/t_par:.2f}x)")
    
    # 場景 2：矩陣運算
    print("\n2. Matrix Operations (already multi-threaded in BLAS):")
    A = np.random.randn(2000, 2000).astype(np.float64)
    B = np.random.randn(2000, 2000).astype(np.float64)
    
    start = time.perf_counter()
    C = A @ B
    t_blas = time.perf_counter() - start
    
    print(f"   BLAS matmul (2000x2000): {t_blas:.3f}s")
    print(f"   (Already using multiple threads internally)")
    
    # 場景 3：CNN 推論
    print("\n3. CNN Inference (100 images):")
    model = SimpleCNN()
    test_imgs = np.random.randn(100, 1, 28, 28).astype(np.float32)
    
    # Batched
    start = time.perf_counter()
    _ = model.forward(test_imgs)
    t_batch = time.perf_counter() - start
    
    print(f"   Batched inference: {t_batch:.4f}s")
    print(f"   (Batching is usually better than multiprocessing for NumPy)")

final_benchmark()

---

## 總結

### Multiprocessing 要點

1. **適用場景**
   - 完全獨立的任務
   - 任務夠重（預處理時間 > 通訊開銷）
   - 不是 numpy 密集運算（BLAS 已經多執行緒）

2. **常用 API**
   - `Pool.map(func, iterable)`: 單參數函數
   - `Pool.starmap(func, iterable)`: 多參數函數
   - `Pool.apply_async`: 非同步執行

3. **效能考量**
   - Pickle/Unpickle 有開銷
   - 進程間通訊有開銷
   - 共享記憶體可減少大資料的傳遞成本

4. **典型應用**
   - 資料載入和預處理
   - 批量檔案處理
   - 超參數搜索

### 優化優先順序（再次強調）

1. 演算法改進 >> 底層優化
2. 向量化 >> Multiprocessing（對於 numpy）
3. Batching >> 單張處理
4. Multiprocessing 用於「真正獨立且夠重」的任務